## Image processing - segmentation and morphological operators

This exercise sheet is based on the "Synchrotron techniques for materials characterization" lecture that was recorded for 26th January 2024. By going through this jupyter notebook and filling in the blanks you will learn how to assess basic properties of your image, perform simple image operations, assess the image quality quantitatively and apply a mean template filter.

The example code and solutions were created by André Lopes Marinho and Berit Zeller-Plumhoff.

### Loading required libraries

You will require the following libraries. If loading any of them fails, please use pip install to install any missing libraries.

In [ ]:
# Importing libraries
import numpy as np
import math
import matplotlib.pyplot as plt
import ipympl
import imageio.v3 as iio
import skimage
from skimage import draw

### Loading images and basic properties

The first task will be to write a function that will automatically load and display an image based on the image path that will be provided as an argument to the function.

Please fill in the missing lines as indicated in the comments to load the image.

In [ ]:
# Jupyter Magic Command that will ensure our images display in our Jupyter 
# document with pixel information that will help us to run commands more efficiently
%matplotlib widget 

def load_and_show_image(path):
    """Loads and shows an image given a defined path.

    Args:
        path(string): String containing path for images with proper file extension. 
            Example: "images/*.tif"

    Returns: 
        numpy.ndarray: Array with image defined in path.
    """
    
    # We will use the skimage library to read our image
    # use the iio.imread function to read in the image and use matplotlib to display in in greyscale
    image = iio.imread(uri=path)
    plt.imshow(image, cmap='gray')
    print(path + ' was loaded successfully.')
    
    return image

Once the image is loaded, we can assess it shape using the .shape command and the type of the image using the .dtype command. Find out what values the commands returns based on different example images that are provided in the test_images folder.

In [ ]:
%matplotlib widget 

img = load_and_show_image(path='test_images/A1_test.png')

Write a short function that prints the image type, height, width and number of channels for any given input image.

In [ ]:
def print_image_props(image):
    """Displays image type, height, width and number of channels given an image.

    Args:
        image(variablename): variable name of an image
    """
    # The method 'iio.imread' returns an numpy.ndarray, so we can perform array operations on the variable
    # 'image', like showing the number of channels/depth of the image.
    image_shape = image.shape
    image_type = image.dtype

    # With this information, we can also print some image properties
    if len(image_shape) > 2:
        print('Type =', image_type)
        print('Height =', image_shape[0])
        print('Width =', image_shape[1])
        print('# of channels =', image_shape[2])
    else:
        print('Type =', image_type)
        print('Height =', image_shape[0])
        print('Width =', image_shape[1])
        print('# of channels =', 1)

In [ ]:
path='test_images/bone_16bit.png'
img = load_and_show_image(path)
print_image_props(img)

### Histogram

In addition to assessing the general image properties, we are usually interested in knowing the greyscale distribution, i.e. the histogram, which gives us a first impression of image quality in terms of contrast and segmentability.

In this following code snipped complete the function to evaluate and the histogram of an image that is input into the function as a numpy.ndarray.

In [ ]:
def show_histogram(image):
    """Showns an histogram given an image. This method considers the image will be
        either grayscale or RGB

    Args:
        img(numpy.ndarray): Image represented by array

    Returns: 
        None
    """
    
    # First, we need to know if the image is grayscale or RGB
    # use the shape function to do so
    image_shape = image.shape
    image_type = image.dtype
    
    # Next, we can define some properties of our histogram figure
    # create an empty figure with title, xlabel, ylabel and set the limits of the x-axis
    plt.figure()
    plt.title("Histogram")
    plt.xlabel("Value")
    plt.ylabel("Pixel count")
    plt.xlim([0, np.max(image)])
        
    # Finally, create a histrogram using the numpy histogram function and plot it
    # use a bin number of 256 in all cases
    if len(image_shape) > 2:
        colors = ("red", "green", "blue")
        for (channel_id, color) in enumerate(colors):
            histogram, bin_edges = np.histogram(image[:, :, channel_id], bins=256, range=(0, np.max(image[:, :, channel_id])))
            plt.plot(bin_edges[0:-1], histogram, color=color)
    else:
        histogram, bin_edges = np.histogram(image, bins=256, range=(0,np.max(image)))
        plt.plot(bin_edges[0:-1], histogram)  
        
    return None

Now load a couple of test images and display their histogram using the functions you have adapted.

In [ ]:
%matplotlib widget 

# Load an image and show its histogram

img = load_and_show_image(path='test_images/bone_16bit.png')
show_histogram(image=img)

### Simple image operations

Once we are able to load an image and assess its properties as well as its histogram, we can perform a number of matrix operations on it.

As shown as an example in the lecture, perform the following operation for the image $A$: $$A_{new}=\frac{A}{4}+100$$ (make sure that the matrix dimensions fit).

Use both the "bone_16bit.png" image and the "A1_test.png" image as input. Comment on how the image operation has changed the original image in both cases.

In [ ]:
%matplotlib widget 

img = load_and_show_image(path='test_images/A1_test.png')

# perform the operation
img_new = (img / 4) + 100 * np.ones((img.shape))

# show the image and print the properties and show the histogram
plt.imshow(img_new, cmap='gray')
print_image_props(img_new)
show_histogram(image=img_new)

Similarly, as shown in the lecture perform the operation $$A_{new}=A*2$$

In [ ]:
%matplotlib widget 

# perform the operation
img_new = (img *2)

# show the image and print the properties and show the histogram
plt.imshow(img_new, cmap='gray')
print_image_props(img)
show_histogram(image=img_new)

In addition to adding or multiplying an image with scalars, we can add/subtract/multiply/divide images. This is a task performed often for example during the flat and dark field correction of projections prior to the 3D reconstruction of images.

In the next task, please perform this correction of the projection "proj.png" in the test_images folder. In the same folder you will find the corresponding "dark" and "flat" image. Compare how the histogram of the projection changes due to the correction and comment on this based on what you observe qualitatively in the image.

In [ ]:
# flat and dark fiel corrections
# load required images

%matplotlib widget 

flat = load_and_show_image(path='test_images/flat.tiff')
dark = load_and_show_image(path='test_images/dark.tiff')
proj = load_and_show_image(path='test_images/proj.tiff')

In [ ]:
# perform correction operation and display the result
%matplotlib widget 

proj_corr=(proj-dark)/(flat-dark)
plt.imshow(proj_corr, cmap='gray')

In [ ]:
# show the histogram of the original and the corrected projection
%matplotlib widget 
show_histogram(image=proj)
show_histogram(image=proj_corr)

### Image quality

In addition to assessing the image histogram, we want to assess the image quality quantitatively. We can do so by calculating the signal-to-noise ratio (SNR) and contrast-to-noise ratio (CNR) for different regions of our image. In the following, you need to add missing code to two functions which will compuate SNR and CNR following the input of two image regions, respectively. We choose the following definitions of SNR and CNR: $$SNR = \frac{S_i}{0.5\sqrt{\sigma_i^2+\sigma_{bg}^2}}$$ and $$CNR = \frac{|S_i-S_j|}{0.5\sqrt{\sigma_i^2+\sigma_j^2}}$$
where $S_i$ and $S_j$ are the mean greyscale values of the regions $i$ and $j$ and the respective $\sigma$ the standard deviation of the greyscale values. For the SNR we attain the overall noise by taking into account the background noise too.

In [ ]:
def get_signal_to_noise_ratio(region_i, region_bg):
    """Calculates Signal-to-Noise Ratio (SNR) by the following expression:
    
                SNR = |mean(region_i)| / 0.5*[sqrt(stdev(region_i)^2 + stdev(region_bg)^2)]                  
                 
    Args:
        region_i(numpy.ndarray): Array slice defining region i
        region_bg(numpy.ndarray): Array slice defining background region
    Returns: 
        double: SNR value
    """
    
    # calculate the mean value of region_i
    a = np.mean(region_i)
    # calculate the denominator of the SNR
    b = 0.5 * math.sqrt(np.std(region_i) ** 2 + np.std(region_bg) ** 2)
    
    # compute and return the SNR
    SNR = a / b
    
    return SNR


def get_contrast_to_noise_ratio(region_i, region_j):
    """Calculates Contrast-to-Noise Ratio (CNR) by the following expression:
    
                CNR = |mean(region_i) - mean(region_j)| / 0.5*[sqrt(stdev(region_i)^2 + stdev(region_j)^2)]                  
                 
    Args:
        region_i(numpy.ndarray): Array slice defining region i
        region_bg(numpy.ndarray): Array slice defining region j
    Returns: 
        double: CNR value
    """
    
    # calculate the numerator of the CNR
    a = abs(np.mean(region_i) - np.mean(region_j))
    # calculate the denominator of the CNR
    b = 0.5 * math.sqrt(np.std(region_i) ** 2 + np.std(region_j) ** 2)
    
    # compute and return the CNR
    CNR = a / b
    
    return CNR

Based on the functions you have just created please calculate both SNR and CNR between the four quadrants of the image SNR_CNR.tif in the folder test_images. Overall, you should create four or six comparisons for SNR and CNR, respectively. To calculate the SNR you may assume that the darkest image quadrant is the background.

In order to use the functions you must identify the regions within the image. Identify the image dimensions to do so.

In [ ]:
# load and display the image

%matplotlib widget 

SNRCNR = load_and_show_image(path='test_images/SNR_CNR.tif')

# print image properties
print_image_props(SNRCNR)

In [ ]:
# define four regions based on the image and its dimensions
%matplotlib widget 


q1=SNRCNR[10:20,0:10]
q2=SNRCNR[10:20,10:20]
q3=SNRCNR[0:10,10:20]
q4=SNRCNR[0:10,0:10]

In [ ]:
# calculate all SNR and CNR

SNR1=get_signal_to_noise_ratio(q1,q1)
SNR2=get_signal_to_noise_ratio(q2,q1)
SNR3=get_signal_to_noise_ratio(q3,q1)
SNR4=get_signal_to_noise_ratio(q4,q1)

CNR12=get_contrast_to_noise_ratio(q1,q2)
CNR13=get_contrast_to_noise_ratio(q1,q3)
CNR14=get_contrast_to_noise_ratio(q1,q4)
CNR23=get_contrast_to_noise_ratio(q2,q3)
CNR24=get_contrast_to_noise_ratio(q2,q4)
CNR34=get_contrast_to_noise_ratio(q3,q4)

# print the computed values
print('SNR of Q1:',np.round(SNR1,2))
print('SNR of Q2:',np.round(SNR2,2))
print('SNR of Q3:',np.round(SNR3,2))
print('SNR of Q4:',np.round(SNR4,2))

print('CNR of Q1 vs Q2:',np.round(CNR12,2))
print('CNR of Q1 vs Q3:',np.round(CNR13,2))
print('CNR of Q1 vs Q4:',np.round(CNR14,2))
print('CNR of Q2 vs Q3:',np.round(CNR23,2))
print('CNR of Q2 vs Q4:',np.round(CNR24,2))
print('CNR of Q3 vs Q4:',np.round(CNR34,2))

Please comment on the values that you have obtained.

In the next task, use image "A1_test2.png" and calculate the CNR for root vs. soil, root vs. background and soil vs. background. Please comment on why you are selecting specific regions and comment on your results.

In [ ]:
%matplotlib widget 

earth = load_and_show_image(path='test_images/A1_test2.png')

print_image_props(earth)

In [ ]:
# Select different regions from the image and calculate SNR and CNR values

# Regions
air = earth[75:100, 185:210]
soil = earth[75:100, 110:135]
root = earth[55:75, 70:90]

# Visualizing them on the image

img_aux = earth.copy()

row_air, col_air = skimage.draw.rectangle_perimeter(start=(75, 185), end=(100, 210))
row_soil, col_soil = skimage.draw.rectangle_perimeter(start=(75, 110), end=(100, 135))
row_root, col_root = skimage.draw.rectangle_perimeter(start=(55, 70), end=(75, 90))

# For better vizualization

fig, ax = plt.subplots()
ax.imshow(img_aux, cmap = 'gray')
ax.plot(col_air, row_air, '-r')
ax.plot(col_soil, row_soil, '-g')
ax.plot(col_root, row_root, '-b')

# Calculate CNR

# CNR - Soil and Root 
CNR1 = get_contrast_to_noise_ratio(soil, root)
print('CNR soil vs. root: ', np.round(CNR1,2))

# CNR - Soil and Air
CNR2 = get_contrast_to_noise_ratio(soil, air)
print('CNR soil vs. air: ', np.round(CNR2,2))

# CNR - Air and Root 
CNR3 = get_contrast_to_noise_ratio(air, root)
print('CNR air vs. root: ', np.round(CNR3,2))

### Image filters

If the image quality that we have determined is insufficient, e.g. if the CNR is particularly low, we can apply filters to improve the noise level while maintaining the sharpness of the features we are interested in. To do so, you will complete a function that defines a mean filter which is applied as a quadratic kernel template to the image. The function takes the original image and the filter kernel size as input and outputs the filtered image. You can move the template across the image in a scanning manner - in order to adjust for the edges correctly, you should apply a zero padding at the image borders.

In [ ]:
def mean_filter(image, filter_size):
    """Applies mean filter in a given image.
                 
    Args:
        image(numpy.ndarray): Image to be filtered
        filter_size(numpy.ndarray): Size of kernel
    Returns: 
        numpy.ndarray: Filtered image
    """
    
    temp = []
    padding = filter_size // 2
    img_final = np.zeros((image.shape))
    
    img_aux=np.zeros((image.shape[0]+padding*2,image.shape[1]+padding*2))
    img_aux[padding:-padding,padding:-padding]=image.copy()
 
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            temp=img_aux[i:i+filter_size,j:j+filter_size]
            img_final[i][j] = np.mean(temp)
            
    return img_final

Apply the filter to the image "A1_test2.png" with different kernel sizes and plot both the original and the filtered image next to each other.

In [ ]:
### %matplotlib widget 

# earth = load_and_show_image(path='test_images/A1_test2.png')

# Mean filter
img_mean = mean_filter(earth, 3)

# Plot your results to compare each one
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(earth, cmap='gray')
ax2.imshow(img_mean, cmap='gray')
fig.tight_layout()

Finally, compute the CNR again for the regions your defined on the image previously and comment on how a mean filter with kernel size 3 has changed the CNR values.

In [ ]:
# Select different regions from the image and calculate SNR and CNR values

# Regions
air2 = img_mean[75:100, 185:210]
soil2 = img_mean[75:100, 110:135]
root2 = img_mean[55:75, 70:90]

# Visualizing them on the image

img_aux = img_mean.copy()

row_air, col_air = skimage.draw.rectangle_perimeter(start=(75, 185), end=(100, 210))
row_soil, col_soil = skimage.draw.rectangle_perimeter(start=(75, 110), end=(100, 135))
row_root, col_root = skimage.draw.rectangle_perimeter(start=(55, 70), end=(75, 90))

# For better vizualization

fig, ax = plt.subplots()
ax.imshow(img_aux, cmap = 'gray')
ax.plot(col_air, row_air, '-r')
ax.plot(col_soil, row_soil, '-g')
ax.plot(col_root, row_root, '-b')

# Calculate CNR

# CNR - Soil and Root 
CNR1_mean = get_contrast_to_noise_ratio(soil2, root2)
print('CNR soil vs. root after filtering: ', np.round(CNR1_mean,2))

# CNR - Soil and Air
CNR2_mean = get_contrast_to_noise_ratio(soil2, air2)
print('CNR soil vs. air after filtering: ', np.round(CNR2_mean,2))

# CNR - Air and Root 
CNR3_mean = get_contrast_to_noise_ratio(air2, root2)
print('CNR air vs. root after filtering: ', np.round(CNR3_mean,2))

In addition to the mean filter, we are interested in using the median and Gaussian filter. Write two functions that will apply the respective filter and output the filtered image, given a certain input image and kernel size.

In [ ]:
%matplotlib widget
def median_filter(image, filter_size):
    """Applies median filter to a given image.
                 
    Args:
        image(numpy.ndarray): Image to be filtered
        filter_size(numpy.ndarray): Size of kernel
        
    Returns: 
        numpy.ndarray: Filtered image      
    """
    temp = []
    padding = filter_size // 2
    img_final = np.zeros((image.shape))
    
    img_aux=np.zeros((image.shape[0]+padding*2,image.shape[1]+padding*2))
    img_aux[padding:-padding,padding:-padding]=image.copy()
 
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            temp=img_aux[i:i+filter_size,j:j+filter_size]
            tmp_aux=np.reshape(temp, (filter_size**2))
            tmp_aux.sort()
            img_final[i][j] = tmp_aux[tmp_aux.shape[0] // 2]
            temp=[]
            
    return img_final

In [ ]:
%matplotlib widget
def gaussian_filter(image, filter_size, sigma):
    """Applies median filter to a given image.
                 
    Args:
        image(numpy.ndarray): Image to be filtered
        filter_size(numpy.ndarray): Size of kernel
        sigma(double): Standard deviation for Gaussian distribution
        
    Returns: 
        numpy.ndarray: Filtered image      
    """
    temp = []
    padding = filter_size // 2
    img_final = np.zeros((image.shape))
    
    kernel = np.fromfunction(lambda x, y: (1 / (2 * math.pi * sigma ** 2)) * \
                             math.e ** ((-1 * ((x - (filter_size - 1) / 2) ** 2 + \
                                               (y - (filter_size - 1) / 2) ** 2)) / (2 * sigma **2)), (filter_size, filter_size))
    
    kernel /= np.sum(kernel)
    
    img_aux=np.zeros((image.shape[0]+padding*2,image.shape[1]+padding*2))
    img_aux[padding:-padding,padding:-padding]=image.copy()
 
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            temp=img_aux[i:i+filter_size,j:j+filter_size]
            temp_aux = np.reshape(temp, (filter_size**2))
            gauss = kernel.reshape((filter_size ** 2))
            img_final[i][j] = np.sum(np.outer(temp_aux, gauss))
            temp=[]
            
    return img_final

Apply the filters you have implemented so far to A1_test2.png using different filter parameters and comment on how the result changes and how the filters compare.

#### Fourier filters

In addition to the above filters, applying a Fourier filter, e.g. either a high-, low- or a bandpass filter can be very useful in order to highlight edges or smoothen the image. In the following, you should write three functions that will apply the different kinds of Fourier filters given an input image and the filter parameters.

Start by loading and displaying A1_test2.png - we will be working with this image in the following.

In [ ]:
%matplotlib widget
image = load_and_show_image(path='test_images/A1_test2.png')

We will begin by applying a highpass filter - you will be required to use the image and the filter size as input to your function. Apply the 2d Fourier transform from numpy to your image and shift the resulting spectrum to the center of the image. You should then create a circular mask based on the filter size that sets all low frequencies, i.e. those in the center of the Fourier spectrum, to zero. Then perform the inverse Fourier operations (shifting the spectrum back first).

Plot the original image, the magnitude spectrum, the highpass filter mask and the image after filtering next to each other.

In [ ]:
def fft_highpass_filter(image, filter_size):
    """Applies highpass Fourier filter to a given image.
                 
    Args:
        image(numpy.ndarray): Image to be filtered
        filter_size(numpy.ndarray): Number of frequency-equivalent pixels to remove
        
    Returns: 
        numpy.ndarray: Filtered image      
    """
    # use the numpy 2D Fourier function to transform the input image
    f = np.fft.fft2(image)
    
    # shift the spectrmu to the center
    fshift = np.fft.fftshift(f)
    magnitude_spectrum = 20*np.log(np.abs(fshift))

    # draw a cylindrical mask using the filter size to filter out the low frequencies
    crow=fshift.shape[0]//2
    ccol=fshift.shape[1]//2
    rr, cc = skimage.draw.disk(center=(crow,ccol), radius=filter_size, shape=fshift.shape[0:2])
    filter_img=np.ones((fshift.shape[0],fshift.shape[1]))
    filter_img[rr, cc] = 0
    fshift = fshift*filter_img
    
    # perform the inverse transform
    f_ishift = np.fft.ifftshift(fshift)
    img_back = np.fft.ifft2(f_ishift)
    img_back = np.abs(img_back)

    # Plot the original image, the magnitude spectrum, the highpass filter mask  
    # and the image after filtering next to each other.
    plt.subplot(141),plt.imshow(image, cmap = 'gray')
    plt.title('Input image'), plt.xticks([]), plt.yticks([])
    plt.subplot(142),plt.imshow(magnitude_spectrum, cmap = 'gray')
    plt.title('Magnitude spectrum'), plt.xticks([]), plt.yticks([])
    plt.subplot(143),plt.imshow(filter_img, cmap = 'gray')
    plt.title('Highpass filter'), plt.xticks([]), plt.yticks([])
    plt.subplot(144),plt.imshow(img_back, cmap = 'gray')
    plt.title('Image after HPF'), plt.xticks([]), plt.yticks([])
    plt.tight_layout()
    plt.show()
               
    return img_back

Now use the function you have just defined on the image of the soil and root with different input filter sizes and comment on how the filter influences the image.

In [ ]:
%matplotlib widget
image_fft=fft_highpass_filter(image, 100)

Next, define a function for a low-pass Fourier filter. Do the same as above, but maintain the low frequencies this time.

In [ ]:
def fft_lowpass_filter(image, filter_size):
    """Applies lowpass Fourier filter to a given image.
                 
    Args:
        image(numpy.ndarray): Image to be filtered
        filter_size(numpy.ndarray): Number of frequency-equivalent pixels to remove
        
    Returns: 
        numpy.ndarray: Filtered image      
    """
    f = np.fft.fft2(image)
    fshift = np.fft.fftshift(f)
    magnitude_spectrum = 20*np.log(np.abs(fshift))

    crow=fshift.shape[0]//2
    ccol=fshift.shape[1]//2
    rr, cc = skimage.draw.disk(center=(crow,ccol), radius=filter_size, shape=fshift.shape[0:2])
    filter_img=np.zeros((fshift.shape[0],fshift.shape[1]))
    filter_img[rr, cc] = 1
    fshift = fshift*filter_img
    f_ishift = np.fft.ifftshift(fshift)
    img_back = np.fft.ifft2(f_ishift)
    img_back = np.abs(img_back)

    plt.subplot(141),plt.imshow(image, cmap = 'gray')
    plt.title('Input image'), plt.xticks([]), plt.yticks([])
    plt.subplot(142),plt.imshow(magnitude_spectrum, cmap = 'gray')
    plt.title('Magnitude spectrum'), plt.xticks([]), plt.yticks([])
    plt.subplot(143),plt.imshow(filter_img, cmap = 'gray')
    plt.title('Lowpass filter'), plt.xticks([]), plt.yticks([])
    plt.subplot(144),plt.imshow(img_back, cmap = 'gray')
    plt.title('Image after LPF'), plt.xticks([]), plt.yticks([])
    plt.tight_layout()
    plt.show()
               
    return img_back

Apply the low-pass filter to the image of the soil and root for different input filter sizes and comment on how the image is changed.

In [ ]:
%matplotlib widget
image_fft=fft_lowpass_filter(image, 10)

Finally, define a Fourier bandpass filter which takes two filter sizes as input for low- and high-frequency thresholds, respectively.

In [ ]:
def fft_bandpass_filter(image, filter_size1, filter_size2):
    """Applies bandpass fourier filter to a given image.
                 
    Args:
        image(numpy.ndarray): Image to be filtered
        filter_size1(numpy.ndarray): Number of frequency-equivalent pixels to remove fp
        
    Returns: 
        numpy.ndarray: Filtered image      
    """
    f = np.fft.fft2(image)
    fshift = np.fft.fftshift(f)
    magnitude_spectrum = 20*np.log(np.abs(fshift))

    crow=fshift.shape[0]//2
    ccol=fshift.shape[1]//2
    rr, cc = skimage.draw.disk(center=(crow,ccol), radius=filter_size2, shape=fshift.shape[0:2])
    filter_img=np.zeros((fshift.shape[0],fshift.shape[1]))
    filter_img[rr, cc] = 1
    rr, cc = skimage.draw.disk(center=(crow,ccol), radius=filter_size1, shape=fshift.shape[0:2])
    #filter_img=np.ones((new_img.shape[0],new_img.shape[1]))
    filter_img[rr, cc] = 0
    fshift = fshift*filter_img
    fshift = fshift*filter_img
    f_ishift = np.fft.ifftshift(fshift)
    img_back = np.fft.ifft2(f_ishift)
    img_back = np.abs(img_back)

    plt.subplot(141),plt.imshow(image, cmap = 'gray')
    plt.title('Input image'), plt.xticks([]), plt.yticks([])
    plt.subplot(142),plt.imshow(magnitude_spectrum, cmap = 'gray')
    plt.title('Magnitude spectrum'), plt.xticks([]), plt.yticks([])
    plt.subplot(143),plt.imshow(filter_img, cmap = 'gray')
    plt.title('Bandpass filter'), plt.xticks([]), plt.yticks([])
    plt.subplot(144),plt.imshow(img_back, cmap = 'gray')
    plt.title('Image after BPF'), plt.xticks([]), plt.yticks([])
    plt.tight_layout()
    plt.show()
               
    return img_back

Apply the bandpass filter to the image of the soil and root for different input filter sizes and comment on how the image is changed.

In [ ]:
%matplotlib widget
image_fft=fft_bandpass_filter(image, 10,20)

### Feature Extraction

In the following, you will become familar with a number of different methods for feature extraction, including thresholding and first and second order edge detection.

#### Thresholding

To set sensible thresholds you should consider the histogram of an image after loading it. Load the the A1_test2.png image, apply a median filter and consider the histogram before and after filtering.

Write a function that will threshold an input image within the range specified as input to the function.

In [ ]:
def threshold(image, low_thresh, up_thresh):
    """Applies a threshold to a given image.
                 
    Args:
        image(numpy.ndarray): Image to be thresholded
        low_thresh(scalar): Value of lower threshold
        up_thresh(scalar): Value of upper threshold
        
    Returns: 
        numpy.ndarray: Thresholded image      
    """

                   
    return img_back

You should see four peaks in the histogram of the filtered image. Apply the thresholding function for each of these peaks and plot the resulting images, as well as the input image, next to each other. Give each image a detail of what it represents.

Play around with the performance of different image filters and kernel sizes and the regions you can subsequently segment using thresholding.

#### First order edge detection

In first order edge detection, the gradient image of the original image is determined and local maxima are then identified. The resulting edge image may either be binary, i.e. already a thresholded version of the local maxima, depending on which step height should be included, or a greyscale image where each greyscale corresponds to the magnitude of the maximum.

Because the process of implementing such a filter is complex, have a look here: https://towardsdatascience.com/canny-edge-detection-step-by-step-in-python-computer-vision-b49c3a2d8123
and try to implement the filter and workflow given there in order to finally retrieve the edges between the soil and the air and the roots and the soil.

### Morphological operators

To easily clean up our image, we can use morphological operators, such as erosion and dilation. In the following, you should implement these as functions that take the binarized image and the kernel size for the (quadratric) operator as input and output the processed image.
Keep in mind that you need to consider the image edges differently than during filtering, i.e. padding may not be appropriate.

#### Erosion operator

First, write a function to apply an erosion operator.

In [ ]:
def erosion_op(image, filter_size):
    """Applies erosion operator to a given binary image.
                 
    Args:
        image(numpy.ndarray): Image to be processed, should contain only 0 and 1
        filter_size(numpy.ndarray): Size of kernel
    Returns: 
        numpy.ndarray: Filtered image
    """
    
    return img_final

Apply the erosion operator to the thresholded image of the root with different kernel sizes and comment on how the image changes.

#### Dilation

Now write a function for a dilation operator.

In [ ]:
def dilation_op(image, filter_size):
    """Applies dilation operator to a given binary image.
                 
    Args:
        image(numpy.ndarray): Image to be processed, should contain only 0 and 1
        filter_size(numpy.ndarray): Size of kernel
    Returns: 
        numpy.ndarray: Filtered image
    """
    
   
    return img_final

Apply the dilation operator to the eroded image of the root with different kernel sizes and comment on how the image changes.

Now plot original image next to the eroded and dilated one, as well as a dilated and eroded one. What differences do you see between the images if you apply kernel sizes of 3 and 5?

### Distance transform

Use the _scipy_ distance transform _distance_transform_ed_ and try to recreate the workflow shown in the lecture.
Starting with the original image "A1_test2.png", filter it, threshold the soil and the root, try to clean the image as musch as possible and apply the distance transform to both images and by multiplying the output obtain the final image where you will have the distance to the root in all soil pixel.
Plot the different intermediate outputs.